# Creating a Multi-AI Deploy App with Multiple Models

This tutorial shows how to create an inference application with multiple models, focusing on model files organization, accessing and inferring with named model network in the application, and finally building an app package.

Typically multiple models will work in tandem, e.g. a lung segmentation model's output, along with the original image, are then used by a lung nodule detection and classification model. There are, however, no such models in the [MONAI Model Zoo](https://github.com/Project-MONAI/model-zoo) as of now. So, for illustration purpose, two independent models will be used in this example, [Spleen Segmentation](https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation) and [Pancreas Segmentation](https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation), both are trained with DICOM images of CT modality, and both are packaged in the [MONAI Bundle](https://docs.monai.io/en/latest/bundle_intro.html) format. A single input of a CT Abdomen DICOM Series can be used for both models within the application.


## Important Steps
- Place the model TorchScripts in a defined folder structure, see below for details
- Pass the model name to the inference operator instance in the app
- Connect the input to and output from the inference operators, as required by the app

## Required Model File Organization

- The model files in TorchScript, be it MONAI Bundle compliant or not, must each be placed in an uniquely named folder. The name of this folder becomes the model network name within the application for it to retrieve the loaded model network from the execution context.
- The folders containing the individual model file must then be place under a parent folder. The name of this folder can be any valid folder name chosen by the application developer.
- The path of the aforementioned parent folder then needs to be used as the value for the well-known environment variable for model path.

## Example Model File Organization

In this example, the models are organized as shown below.
```
multi_models
├── pancreas_ct_dints
│   └── model.ts
└── spleen_ct
    └── model.ts
```

Please note,

- The `multi_models` is the parent folder, whose path is used as the value for setting the well-known environment variable for model path, and when using App SDK CLI to build the application package.
- The sub-folder names become model network names, `pancreas_ct_dints` and `spleen_model`, respectively.

In the following sections, we will demonstrate how to create and package the application using these two models.

:::{note}
The two models are both MONAI bundles, published in [MONAI Model Zoo](https://github.com/Project-MONAI/model-zoo)
- [spleen_ct_segmentation, v0.3.2](https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation)
- [pancreas_ct_dints_segmentation, v0.3.0](https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation)

The DICOM CT series used as test input is downloaded from [TCIA](https://www.cancerimagingarchive.net/), CT Abdomen Collection ID `CPTAC-PDA` Subject ID `C3N-00198`.

Both the DICOM files and the models have been packaged and shared on Google Drive.
:::

## Creating Operators and connecting them in Application class

We will implement an application that consists of seven Operators:

- **DICOMDataLoaderOperator**:
    - **Input(dicom_files)**: a folder path (`Path`)
    - **Output(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
- **DICOMSeriesSelectorOperator**:
    - **Input(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
    - **Input(selection_rules)**: a selection rule (Dict)
    - **Output(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
- **DICOMSeriesToVolumeOperator**:
    - **Input(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
    - **Output(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **MonaiBundleInferenceOperator** x 2:
    - **Input(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output(pred)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **DICOMSegmentationWriterOperator** x2:
    - **Input(seg_image)**: a segmentation image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Input(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
    - **Output(dicom_seg_instance)**: a file path (`Path`)


:::{note}
The `DICOMSegmentationWriterOperator` needs both the segmentation image as well as the original DICOM series for reusing the patient demographics and other DICOM Study level attributes, as well as referring to the original SOP instance UID.
:::

The workflow of the application is illustrated below.

```{mermaid}
%%{init: {"theme": "base", "themeVariables": { "fontSize": "16px"}} }%%

classDiagram
    direction TB
    DICOMDataLoaderOperator --|> DICOMSeriesSelectorOperator : dicom_study_list...dicom_study_list
    DICOMSeriesSelectorOperator --|> DICOMSeriesToVolumeOperator : study_selected_series_list...study_selected_series_list

    DICOMSeriesToVolumeOperator --|> Spleen_BundleInferenceOperator : image...image
    DICOMSeriesSelectorOperator --|> Spleen_DICOMSegmentationWriterOperator : study_selected_series_list...study_selected_series_list
    Spleen_BundleInferenceOperator --|> Spleen_DICOMSegmentationWriterOperator : pred...seg_image

    DICOMSeriesToVolumeOperator --|> Pancreas_BundleInferenceOperator : image...image
    DICOMSeriesSelectorOperator --|> Pancreas_DICOMSegmentationWriterOperator : study_selected_series_list...study_selected_series_list
    Pancreas_BundleInferenceOperator --|> Pancreas_DICOMSegmentationWriterOperator : pred...seg_image

    class DICOMDataLoaderOperator {
        <in>dicom_files : DISK
        dicom_study_list(out) IN_MEMORY
    }
    class DICOMSeriesSelectorOperator {
        <in>dicom_study_list : IN_MEMORY
        <in>selection_rules : IN_MEMORY
        study_selected_series_list(out) IN_MEMORY
    }
    class DICOMSeriesToVolumeOperator {
        <in>study_selected_series_list : IN_MEMORY
        image(out) IN_MEMORY
    }
    class Spleen_BundleInferenceOperator {
        <in>image : IN_MEMORY
        pred(out) IN_MEMORY
    }
    class Pancreas_BundleInferenceOperator {
        <in>image : IN_MEMORY
        pred(out) IN_MEMORY
    }
    class Spleen_DICOMSegmentationWriterOperator {
        <in>seg_image : IN_MEMORY
        <in>study_selected_series_list : IN_MEMORY
        dicom_seg_instance(out) DISK
    }
    class Pancreas_DICOMSegmentationWriterOperator {
        <in>seg_image : IN_MEMORY
        <in>study_selected_series_list : IN_MEMORY
        dicom_seg_instance(out) DISK
    }
```

### Setup environment


In [1]:
# Install MONAI and other necessary image processing packages for the application
!python -c "import monai" || pip install --upgrade -q "monai"
!python -c "import torch" || pip install -q "torch>=1.12.0"
!python -c "import numpy" || pip install -q "numpy>=1.21"
!python -c "import nibabel" || pip install -q "nibabel>=3.2.1"
!python -c "import pydicom" || pip install -q "pydicom>=1.4.2"
!python -c "import highdicom" || pip install -q "highdicom>=0.18.2"
!python -c "import SimpleITK" || pip install -q "SimpleITK>=2.0.0"
!python -c "import typeguard" || pip install -q "typeguard~=2.12.1"

# Install MONAI Deploy App SDK package
!python -c "import monai.deploy" || pip install --upgrade -q "monai-deploy-app-sdk"

Note: you may need to restart the Jupyter kernel to use the updated packages.

### Download/Extract input and model/bundle files from Google Drive

In [2]:
# Download the test data and MONAI bundle zip file
!pip install gdown 
!gdown "https://drive.google.com/uc?id=1llJ4NGNTjY187RLX4MtlmHYhfGxBNWmd"

# After downloading ai_spleen_bundle_data zip file from the web browser or using gdown,
!unzip -o "ai_multi_model_bundle_data.zip"

Downloading...
From: https://drive.google.com/uc?id=1llJ4NGNTjY187RLX4MtlmHYhfGxBNWmd
To: /home/mqin/src/monai-deploy-app-sdk/notebooks/tutorials/ai_multi_model_bundle_data.zip
100%|████████████████████████████████████████| 647M/647M [00:07<00:00, 82.1MB/s]
Archive:  ai_multi_model_bundle_data.zip
  inflating: dcm/1-001.dcm           
  inflating: dcm/1-002.dcm           
  inflating: dcm/1-003.dcm           
  inflating: dcm/1-004.dcm           
  inflating: dcm/1-005.dcm           
  inflating: dcm/1-006.dcm           
  inflating: dcm/1-007.dcm           
  inflating: dcm/1-008.dcm           
  inflating: dcm/1-009.dcm           
  inflating: dcm/1-010.dcm           
  inflating: dcm/1-011.dcm           
  inflating: dcm/1-012.dcm           
  inflating: dcm/1-013.dcm           
  inflating: dcm/1-014.dcm           
  inflating: dcm/1-015.dcm           
  inflating: dcm/1-016.dcm           
  inflating: dcm/1-017.dcm           
  inflating: dcm/1-018.dcm           
  inflating: dcm/

### Set up environment variables

In [3]:
%env HOLOSCAN_INPUT_PATH dcm
%env HOLOSCAN_MODEL_PATH multi_models
%env HOLOSCAN_OUTPUT_PATH output
%ls $HOLOSCAN_INPUT_PATH

env: HOLOSCAN_INPUT_PATH=dcm
env: HOLOSCAN_MODEL_PATH=multi_models
env: HOLOSCAN_OUTPUT_PATH=output
1-001.dcm  1-031.dcm  1-061.dcm  1-091.dcm  1-121.dcm  1-151.dcm  1-181.dcm
1-002.dcm  1-032.dcm  1-062.dcm  1-092.dcm  1-122.dcm  1-152.dcm  1-182.dcm
1-003.dcm  1-033.dcm  1-063.dcm  1-093.dcm  1-123.dcm  1-153.dcm  1-183.dcm
1-004.dcm  1-034.dcm  1-064.dcm  1-094.dcm  1-124.dcm  1-154.dcm  1-184.dcm
1-005.dcm  1-035.dcm  1-065.dcm  1-095.dcm  1-125.dcm  1-155.dcm  1-185.dcm
1-006.dcm  1-036.dcm  1-066.dcm  1-096.dcm  1-126.dcm  1-156.dcm  1-186.dcm
1-007.dcm  1-037.dcm  1-067.dcm  1-097.dcm  1-127.dcm  1-157.dcm  1-187.dcm
1-008.dcm  1-038.dcm  1-068.dcm  1-098.dcm  1-128.dcm  1-158.dcm  1-188.dcm
1-009.dcm  1-039.dcm  1-069.dcm  1-099.dcm  1-129.dcm  1-159.dcm  1-189.dcm
1-010.dcm  1-040.dcm  1-070.dcm  1-100.dcm  1-130.dcm  1-160.dcm  1-190.dcm
1-011.dcm  1-041.dcm  1-071.dcm  1-101.dcm  1-131.dcm  1-161.dcm  1-191.dcm
1-012.dcm  1-042.dcm  1-072.dcm  1-102.dcm  1-132.dcm  1-162.dcm

### Set up imports

Let's import necessary classes/decorators to define Application and Operator.

In [4]:

import logging
from pathlib import Path

# Required for setting SegmentDescription attributes. Direct import as this is not part of App SDK package.
from pydicom.sr.codedict import codes

from monai.deploy.conditions import CountCondition
from monai.deploy.core import AppContext, Application
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.logger import load_env_log_level
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import (
    BundleConfigNames,
    IOMapping,
    MonaiBundleInferenceOperator,
)
from monai.deploy.operators.stl_conversion_operator import STLConversionOperator


### Determining the Model Name and I/O for the Model Bundle Inference Operator

The App SDK provides a `MonaiBundleInferenceOperator` class to perform inference with a MONAI Bundle, which is essentially a PyTorch model in TorchScript with additional metadata describing the model network and processing specification. This operator uses the MONAI utilities to parse a MONAI Bundle to automatically instantiate the objects required for input and output processing as well as inference, as such it depends on MONAI transforms, inferers, and in turn their dependencies.

Each Operator class inherits from the base `Operator` class, and its named input/output properties are specified by using the function `setup`. For the `MonaiBundleInferenceOperator` class, the input/output need to be defined to match those of the model network, both in name and data type. For the current release, an `IOMapping` object is used to connect the operator input/output to those of the model network by matching the names. This is subject to change in the future releases.

When multiple models are used in an application, it is important that each model network name is passed in when creating the inference operator, via the `model_name` argument.

Both the Spleen CT Segmentation and Pancreas model networks have a named input, called "image", and a named output called "pred", both are of image type. These can all be mapped to the App SDK [Image](/modules/_autosummary/monai.deploy.core.domain.Image). The information on model network input and output data types is typically acquired by examining the model metadata `network_data_format` in the MONAI Bundle, as seen in this [example] (https://github.com/Project-MONAI/model-zoo/blob/dev/models/spleen_ct_segmentation/configs/metadata.json).

### Creating Application class

Our application class would look like below.

It defines `App` class, inheriting `Application` class.

Objects required for DICOM parsing, series selection, pixel data conversion to volume image, model specific inference, and the AI result specific DICOM Segmentation object writers are created. The execution pipeline, as a Directed Acyclic Graph, is then constructed by connecting these objects through function `add_flow()`.

In [5]:
class App(Application):
    """This example demonstrates how to create a multi-model/multi-AI application.

    The important steps are:
        1. Place the model TorchScripts in a defined folder structure, see below for details
        2. Pass the model name to the inference operator instance in the app
        3. Connect the input to and output from the inference operators, as required by the app

    Required Model Folder Structure:
        1. The model TorchScripts, be it MONAI Bundle compliant or not, must be placed in
           a parent folder, whose path is used as the path to the model(s) on app execution
        2. Each TorchScript file needs to be in a sub-folder, whose name is the model name

    An example is shown below, where the `parent_foler` name can be the app's own choosing, and
    the sub-folder names become model names, `pancreas_ct_dints` and `spleen_model`, respectively.

        <parent_fodler>
        ├── pancreas_ct_dints
        │   └── model.ts
        └── spleen_ct
            └── model.ts

    Note:
    1. The TorchScript files of MONAI Bundles can be downloaded from MONAI Model Zoo, at
       https://github.com/Project-MONAI/model-zoo/tree/dev/models
       https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation, v0.3.2
       https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation, v0.3.8
    2. The input DICOM instances are from a DICOM Series of CT Abdomen, similar to the ones
       used in the Spleen Segmentation example
    3. This example is purely for technical demonstration, not for clinical use

    Execution Time Estimate:
      With a Nvidia GV100 32GB GPU, the execution time is around 87 seconds for an input DICOM series of 204 instances,
      and 167 second for a series of 515 instances.
    """

    def __init__(self, *args, **kwargs):
        """Creates an application instance."""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # This method calls the base class to run. Can be omitted if simply calling through.
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """Creates the app specific operators and chain them up in the processing DAG."""

        logging.info(f"Begin {self.compose.__name__}")

        app_context = AppContext({})  # Let it figure out all the attributes without overriding
        app_input_path = Path(app_context.input_path)
        app_output_path = Path(app_context.output_path)

        # Create the custom operator(s) as well as SDK built-in operator(s).
        study_loader_op = DICOMDataLoaderOperator(
            self, CountCondition(self, 1), input_folder=app_input_path, name="study_loader_op"
        )
        series_selector_op = DICOMSeriesSelectorOperator(self, rules=Sample_Rules_Text, name="series_selector_op")
        series_to_vol_op = DICOMSeriesToVolumeOperator(self, name="series_to_vol_op")

        # Create the inference operator that supports MONAI Bundle and automates the inference.
        # The IOMapping labels match the input and prediction keys in the pre and post processing.
        # The model_name needs to be provided as this is a multi-model application and each inference
        # operator need to rely on the name to access the named loaded model network.
        # create an inference operator for each.
        #
        # Pertinent MONAI Bundle:
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation, v0.3.2
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation, v0.3.8

        config_names = BundleConfigNames(config_names=["inference"])  # Same as the default

        # This is the inference operator for the spleen_model bundle. Note the model name.
        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            self,
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            app_context=app_context,
            bundle_config_names=config_names,
            model_name="spleen_ct",
            name="bundle_spleen_seg_op",
        )

        # This is the inference operator for the pancreas_ct_dints bundle. Note the model name.
        bundle_pancreas_seg_op = MonaiBundleInferenceOperator(
            self,
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            app_context=app_context,
            bundle_config_names=config_names,
            model_name="pancreas_ct_dints",
            name="bundle_pancreas_seg_op",
        )

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue. The segment_label, algorithm_name,
        # and algorithm_version are of DICOM VR LO type, limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        #
        # NOTE: Each generated DICOM Seg will be a dcm file with the name based on the SOP instance UID.

        # Description for the Spleen seg, and the seg writer obj
        seg_descriptions_spleen = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.3.2",
            )
        ]

        custom_tags_spleen = {"SeriesDescription": "AI Spleen Seg for research use only. Not for clinical use."}
        dicom_seg_writer_spleen = DICOMSegmentationWriterOperator(
            self,
            segment_descriptions=seg_descriptions_spleen,
            custom_tags=custom_tags_spleen,
            output_folder=app_output_path,
            name="dicom_seg_writer_spleen",
        )

        # Description for the Pancreas seg, and the seg writer obj
        _algorithm_name = "Pancreas CT DiNTS segmentation from CT image"
        _algorithm_family = codes.DCM.ArtificialIntelligence
        _algorithm_version = "0.3.8"

        seg_descriptions_pancreas = [
            SegmentDescription(
                segment_label="Pancreas",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Pancreas,
                algorithm_name=_algorithm_name,
                algorithm_family=_algorithm_family,
                algorithm_version=_algorithm_version,
            ),
            SegmentDescription(
                segment_label="Tumor",
                segmented_property_category=codes.SCT.Tumor,
                segmented_property_type=codes.SCT.Tumor,
                algorithm_name=_algorithm_name,
                algorithm_family=_algorithm_family,
                algorithm_version=_algorithm_version,
            ),
        ]
        custom_tags_pancreas = {"SeriesDescription": "AI Pancreas Seg for research use only. Not for clinical use."}

        dicom_seg_writer_pancreas = DICOMSegmentationWriterOperator(
            self,
            segment_descriptions=seg_descriptions_pancreas,
            custom_tags=custom_tags_pancreas,
            output_folder=app_output_path,
            name="dicom_seg_writer_pancreas",
        )

        # NOTE: Sharp eyed readers can already see that the above instantiation of object can be simply parameterized.
        #       Very true, but leaving them as if for easy reading. In fact the whole app can be parameterized for general use.

        # Create the processing pipeline, by specifying the upstream and downstream operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        self.add_flow(study_loader_op, series_selector_op, {("dicom_study_list", "dicom_study_list")})
        self.add_flow(
            series_selector_op, series_to_vol_op, {("study_selected_series_list", "study_selected_series_list")}
        )

        # Feed the input image to all inference operators
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {("image", "image")})
        # The Pancreas CT Seg bundle requires PyTorch 1.12.0 to avoid failure to load.
        self.add_flow(series_to_vol_op, bundle_pancreas_seg_op, {("image", "image")})

        # Create DICOM Seg for one of the inference output
        # Note below the dicom_seg_writer requires two inputs, each coming from a upstream operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer_spleen, {("study_selected_series_list", "study_selected_series_list")}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer_spleen, {("pred", "seg_image")})

        # Create DICOM Seg for one of the inference output
        # Note below the dicom_seg_writer requires two inputs, each coming from a upstream operator.
        self.add_flow(
            series_selector_op,
            dicom_seg_writer_pancreas,
            {("study_selected_series_list", "study_selected_series_list")},
        )
        self.add_flow(bundle_pancreas_seg_op, dicom_seg_writer_pancreas, {("pred", "seg_image")})

        logging.info(f"End {self.compose.__name__}")


# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""

## Executing app locally

We can execute the app in the Jupyter notebook. Note that the DICOM files of the CT Abdomen series must be present in the input folder, the models are already staged, and and environment variables are set.


In [6]:
app = App()

app.run()

2023-05-15 23:57:59,744 - Begin run
2023-05-15 23:57:59.706 INFO  /workspace/holoscan-sdk/src/core/executors/gxf/gxf_executor.cpp@71: Creating context
2023-05-15 23:57:59,746 - Begin compose
2023-05-15 23:57:59,755 - End compose
2023-05-15 23:57:59.758 INFO  gxf/std/greedy_scheduler.cpp@184: Scheduling 9 entities
2023-05-15 23:57:59,759 - No or invalid input path from the optional input port: None


[2023-05-15 23:57:59.746] [holoscan] [info] [gxf_executor.cpp:100] Loading extensions from configs...
[2023-05-15 23:57:59.756] [holoscan] [info] [gxf_executor.cpp:291] Activating Graph...
[2023-05-15 23:57:59.758] [holoscan] [info] [gxf_executor.cpp:293] Running Graph...
[2023-05-15 23:57:59.758] [holoscan] [info] [gxf_executor.cpp:295] Waiting for completion...


2023-05-15 23:58:00,306 - Finding series for Selection named: CT Series
2023-05-15 23:58:00,307 - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
2023-05-15 23:58:00,308 - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
2023-05-15 23:58:00,308 - On attribute: 'StudyDescription' to match value: '(.*?)'
2023-05-15 23:58:00,309 -     Series attribute StudyDescription value: CT ABDOMEN W IV CONTRAST
2023-05-15 23:58:00,309 - Series attribute string value did not match. Try regEx.
2023-05-15 23:58:00,310 - On attribute: 'Modality' to match value: '(?i)CT'
2023-05-15 23:58:00,311 -     Series attribute Modality value: CT
2023-05-15 23:58:00,311 - Series attribute string value did not match. Try regEx.
2023-05-15 23:58:00,312 - On attribute: 'SeriesDescription' to match value: '(.*?)'
2023-05-15 23:58:00,312 -     Series attribute SeriesDescription value: ABD/PANC 3.0 B31f
2023-05-15 23:58:0

/home/mqin/src/monai-deploy-app-sdk/.venv/lib/python3.8/site-packages/highdicom/valuerep.py:54: UserWarning: The string "C3N-00198" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  warnings.warn(


2023-05-15 23:59:33,218 - add plane #0 for segment #1
2023-05-15 23:59:33,221 - add plane #1 for segment #1
2023-05-15 23:59:33,223 - add plane #2 for segment #1
2023-05-15 23:59:33,224 - add plane #3 for segment #1
2023-05-15 23:59:33,226 - add plane #4 for segment #1
2023-05-15 23:59:33,227 - add plane #5 for segment #1
2023-05-15 23:59:33,229 - add plane #6 for segment #1
2023-05-15 23:59:33,231 - add plane #7 for segment #1
2023-05-15 23:59:33,232 - add plane #8 for segment #1
2023-05-15 23:59:33,234 - add plane #9 for segment #1
2023-05-15 23:59:33,235 - add plane #10 for segment #1
2023-05-15 23:59:33,237 - add plane #11 for segment #1
2023-05-15 23:59:33,238 - add plane #12 for segment #1
2023-05-15 23:59:33,240 - add plane #13 for segment #1
2023-05-15 23:59:33,241 - add plane #14 for segment #1
2023-05-15 23:59:33,243 - add plane #15 for segment #1
2023-05-15 23:59:33,244 - add plane #16 for segment #1
2023-05-15 23:59:33,246 - add plane #17 for segment #1
2023-05-15 23:59:33,

[2023-05-15 23:59:35.087] [holoscan] [info] [gxf_executor.cpp:297] Deactivating Graph...


Once the application is verified inside Jupyter notebook, we can write the whole application as a file, adding the following lines:

```python
if __name__ == "__main__":
    App().run()
```

The above lines are needed to execute the application code by using `python` interpreter.

A `__main__.py` file should also be added, so the application folder structure would look like below:

```bash
my_app
├── __main__.py
└── app.py
```

In [7]:
# Create an application folder
!mkdir -p my_app

### app.py

In [8]:
%%writefile my_app/app.py
# Copyright 2021-2023 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging
from pathlib import Path

# Required for setting SegmentDescription attributes. Direct import as this is not part of App SDK package.
from pydicom.sr.codedict import codes

from monai.deploy.conditions import CountCondition
from monai.deploy.core import AppContext, Application
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.logger import load_env_log_level
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import (
    BundleConfigNames,
    IOMapping,
    MonaiBundleInferenceOperator,
)


class App(Application):
    """This example demonstrates how to create a multi-model/multi-AI application.

    The important steps are:
        1. Place the model TorchScripts in a defined folder structure, see below for details
        2. Pass the model name to the inference operator instance in the app
        3. Connect the input to and output from the inference operators, as required by the app

    Required Model Folder Structure:
        1. The model TorchScripts, be it MONAI Bundle compliant or not, must be placed in
           a parent folder, whose path is used as the path to the model(s) on app execution
        2. Each TorchScript file needs to be in a sub-folder, whose name is the model name

    An example is shown below, where the `parent_foler` name can be the app's own choosing, and
    the sub-folder names become model names, `pancreas_ct_dints` and `spleen_model`, respectively.

        <parent_fodler>
        ├── pancreas_ct_dints
        │   └── model.ts
        └── spleen_ct
            └── model.ts

    Note:
    1. The TorchScript files of MONAI Bundles can be downloaded from MONAI Model Zoo, at
       https://github.com/Project-MONAI/model-zoo/tree/dev/models
       https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation, v0.3.2
       https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation, v0.3.8
    2. The input DICOM instances are from a DICOM Series of CT Abdomen, similar to the ones
       used in the Spleen Segmentation example
    3. This example is purely for technical demonstration, not for clinical use

    Execution Time Estimate:
      With a Nvidia GV100 32GB GPU, the execution time is around 87 seconds for an input DICOM series of 204 instances,
      and 167 second for a series of 515 instances.
    """

    def __init__(self, *args, **kwargs):
        """Creates an application instance."""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # This method calls the base class to run. Can be omitted if simply calling through.
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """Creates the app specific operators and chain them up in the processing DAG."""

        logging.info(f"Begin {self.compose.__name__}")

        app_context = AppContext({})  # Let it figure out all the attributes without overriding
        app_input_path = Path(app_context.input_path)
        app_output_path = Path(app_context.output_path)

        # Create the custom operator(s) as well as SDK built-in operator(s).
        study_loader_op = DICOMDataLoaderOperator(
            self, CountCondition(self, 1), input_folder=app_input_path, name="study_loader_op"
        )
        series_selector_op = DICOMSeriesSelectorOperator(self, rules=Sample_Rules_Text, name="series_selector_op")
        series_to_vol_op = DICOMSeriesToVolumeOperator(self, name="series_to_vol_op")

        # Create the inference operator that supports MONAI Bundle and automates the inference.
        # The IOMapping labels match the input and prediction keys in the pre and post processing.
        # The model_name needs to be provided as this is a multi-model application and each inference
        # operator need to rely on the name to access the named loaded model network.
        # create an inference operator for each.
        #
        # Pertinent MONAI Bundle:
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation, v0.3.2
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation, v0.3.8

        config_names = BundleConfigNames(config_names=["inference"])  # Same as the default

        # This is the inference operator for the spleen_model bundle. Note the model name.
        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            self,
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            app_context=app_context,
            bundle_config_names=config_names,
            model_name="spleen_ct",
            name="bundle_spleen_seg_op",
        )

        # This is the inference operator for the pancreas_ct_dints bundle. Note the model name.
        bundle_pancreas_seg_op = MonaiBundleInferenceOperator(
            self,
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            app_context=app_context,
            bundle_config_names=config_names,
            model_name="pancreas_ct_dints",
            name="bundle_pancreas_seg_op",
        )

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue. The segment_label, algorithm_name,
        # and algorithm_version are of DICOM VR LO type, limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        #
        # NOTE: Each generated DICOM Seg will be a dcm file with the name based on the SOP instance UID.

        # Description for the Spleen seg, and the seg writer obj
        seg_descriptions_spleen = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.3.2",
            )
        ]

        custom_tags_spleen = {"SeriesDescription": "AI Spleen Seg for research use only. Not for clinical use."}
        dicom_seg_writer_spleen = DICOMSegmentationWriterOperator(
            self,
            segment_descriptions=seg_descriptions_spleen,
            custom_tags=custom_tags_spleen,
            output_folder=app_output_path,
            name="dicom_seg_writer_spleen",
        )

        # Description for the Pancreas seg, and the seg writer obj
        _algorithm_name = "Pancreas CT DiNTS segmentation from CT image"
        _algorithm_family = codes.DCM.ArtificialIntelligence
        _algorithm_version = "0.3.8"

        seg_descriptions_pancreas = [
            SegmentDescription(
                segment_label="Pancreas",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Pancreas,
                algorithm_name=_algorithm_name,
                algorithm_family=_algorithm_family,
                algorithm_version=_algorithm_version,
            ),
            SegmentDescription(
                segment_label="Tumor",
                segmented_property_category=codes.SCT.Tumor,
                segmented_property_type=codes.SCT.Tumor,
                algorithm_name=_algorithm_name,
                algorithm_family=_algorithm_family,
                algorithm_version=_algorithm_version,
            ),
        ]
        custom_tags_pancreas = {"SeriesDescription": "AI Pancreas Seg for research use only. Not for clinical use."}

        dicom_seg_writer_pancreas = DICOMSegmentationWriterOperator(
            self,
            segment_descriptions=seg_descriptions_pancreas,
            custom_tags=custom_tags_pancreas,
            output_folder=app_output_path,
            name="dicom_seg_writer_pancreas",
        )

        # NOTE: Sharp eyed readers can already see that the above instantiation of object can be simply parameterized.
        #       Very true, but leaving them as if for easy reading. In fact the whole app can be parameterized for general use.

        # Create the processing pipeline, by specifying the upstream and downstream operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        self.add_flow(study_loader_op, series_selector_op, {("dicom_study_list", "dicom_study_list")})
        self.add_flow(
            series_selector_op, series_to_vol_op, {("study_selected_series_list", "study_selected_series_list")}
        )

        # Feed the input image to all inference operators
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {("image", "image")})
        # The Pancreas CT Seg bundle requires PyTorch 1.12.0 to avoid failure to load.
        self.add_flow(series_to_vol_op, bundle_pancreas_seg_op, {("image", "image")})

        # Create DICOM Seg for one of the inference output
        # Note below the dicom_seg_writer requires two inputs, each coming from a upstream operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer_spleen, {("study_selected_series_list", "study_selected_series_list")}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer_spleen, {("pred", "seg_image")})

        # Create DICOM Seg for one of the inference output
        # Note below the dicom_seg_writer requires two inputs, each coming from a upstream operator.
        self.add_flow(
            series_selector_op,
            dicom_seg_writer_pancreas,
            {("study_selected_series_list", "study_selected_series_list")},
        )
        self.add_flow(bundle_pancreas_seg_op, dicom_seg_writer_pancreas, {("pred", "seg_image")})

        logging.info(f"End {self.compose.__name__}")


# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""

if __name__ == "__main__":
    load_env_log_level()
    logging.info(f"Begin {__name__}")
    App().run()
    logging.info(f"End {__name__}")

Overwriting my_app/app.py


In [9]:
%%writefile my_app/__main__.py
import logging

from app import App

from monai.deploy.logger import load_env_log_level

if __name__ == "__main__":
    load_env_log_level()
    logging.info(f"Begin {__name__}")
    App().run()
    logging.info(f"End {__name__}")

Overwriting my_app/__main__.py


In [10]:
!ls my_app

app.py	__main__.py  __pycache__


At this time, let's execute the app on the command line. Note the required environment variables have been set with the specific input data and model paths from earlier steps.

In [11]:
!rm -f output/*
!python my_app

2023-05-15 23:59:41,314 - Begin __main__
2023-05-15 23:59:41.314 INFO  /workspace/holoscan-sdk/src/core/executors/gxf/gxf_executor.cpp@71: Creating context
2023-05-15 23:59:41,338 - Begin run
[2023-05-15 23:59:41.338] [holoscan] [info] [gxf_executor.cpp:100] Loading extensions from configs...
2023-05-15 23:59:41,338 - Begin compose
2023-05-15 23:59:41,346 - End compose
[2023-05-15 23:59:41.346] [holoscan] [info] [gxf_executor.cpp:291] Activating Graph...
[2023-05-15 23:59:41.347] [holoscan] [info] [gxf_executor.cpp:293] Running Graph...
[2023-05-15 23:59:41.347] [holoscan] [info] [gxf_executor.cpp:295] Waiting for completion...
2023-05-15 23:59:41.347 INFO  gxf/std/greedy_scheduler.cpp@184: Scheduling 9 entities
2023-05-15 23:59:41,348 - No or invalid input path from the optional input port: None
2023-05-15 23:59:41,727 - Finding series for Selection named: CT Series
2023-05-15 23:59:41,727 - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of ser

In [12]:
!ls output

1.2.826.0.1.3680043.10.511.3.11577191598882031081604749632873012.dcm
1.2.826.0.1.3680043.10.511.3.97167693944598203077405429222203356.dcm


## Packaging app

Let's package the app with [MONAI Application Packager](/developing_with_sdk/packaging_app).

In [13]:
!echo "Pending completion of holoscan packager"

Pending completion of holoscan packager


:::{note}
Building a MONAI Application Package (Docker image) can take time. Use `-l DEBUG` option if you want to see the progress.
:::

We can see that the Docker image is created.

In [14]:
!docker image ls | grep my_app

my_app                                                                latest                                     3eb9c4bdc337   8 weeks ago     15GB


## Executing packaged app locally

The packaged app can be run locally through [MONAI Application Runner](/developing_with_sdk/executing_packaged_app_locally).

In [15]:
!echo "Pending completion of Holoscan runner"

Pending completion of Holoscan runner


In [16]:
!ls output

1.2.826.0.1.3680043.10.511.3.11577191598882031081604749632873012.dcm
1.2.826.0.1.3680043.10.511.3.97167693944598203077405429222203356.dcm
